# Research Notebook
## Maxim Astaschov (Goethe University)
## Date: 10.03.2025

# 1: Experience
## Describe at least one research activity you worked on this week. 

- Read through paper Adhikari et al. 2014 [2]
- Added the fitting function from [2] to colossus code [1]
- Worked through necessery steps to implement the complete model

## Motivation:

The goal is to generalize the model in Adhikari et al. 2014 [2] to be used outside of cold dark matter.

# 2: What? (What happened?)
## Describe what happened during your activities for the week.
1. Set up a test environment for a new model, which plots both models and also the difference between them
2. Implemented a new model into the colossus code, which is a simple copy of the Adhikari14 model already existing there
3. Generated the s and Om values as bins and compared to the LUT -> 0m is linspace while s is geomspace
4. Changed the intervals to the given intervals in [2]
5. Implemented Delta_sp in the form of the given fitting function in [2]
6. Let the test environment run both models -> They differ by around 6% in some parts, which is slightly more than the promised ~5% in [2]
7. Could not find an error in the code itself and looked up possible ways to implement the model itself
```
#Test setup:
import numpy as np
import matplotlib

matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from colossus.cosmology import cosmology
from colossus.halo import splashback
from colossus.lss import peaks

for model_name in splashback.models:
    print(model_name)

def main():
    cosmology.setCosmology('planck18')
    z = 1.0
    M200m = 1E12
    nu200m = peaks.peakHeight(M200m, z)
    Gamma = np.arange(0.0, 4.0, 0.5)

    plt.figure()
    plt.xlabel('Accretion rate')
    plt.ylabel('Rsp/R200m')

    RspR200m_ad14, mask_ad14 = splashback.splashbackModel('RspR200m', Gamma=Gamma, nu200m=nu200m, z=z,
                                                model="adhikari14", rspdef='sp-apr-p75', statistic='median')
    plt.plot(Gamma[mask_ad14], RspR200m_ad14, label="adhikari14".replace('_', '\\_'))

    RspR200m_ad14gen, mask_ad14gen = splashback.splashbackModel('RspR200m', Gamma=Gamma, nu200m=nu200m, z=z,
                                                model="adhikari14Generalized", rspdef='sp-apr-p75', statistic='median')
    plt.plot(Gamma[mask_ad14gen], RspR200m_ad14gen, label="adhikari14Generalized".replace('_', '\\_'))

    RspR200m_diff = RspR200m_ad14 - RspR200m_ad14gen
    plt.plot(Gamma[mask_ad14gen], RspR200m_diff, label="Difference between models".replace('_', '\\_'))
    print("Difference in %: ", abs(100*RspR200m_diff/RspR200m_ad14gen))
    plt.legend()
    plt.show()

main()

```
Output:
```
adhikari14Generalized
adhikari14
more15
shi16
mansfield17
diemer17
diemer20
Difference in %:  [1.50507702 3.62257011 5.8075424  6.30313469 4.95862271 1.11194448
 6.18126625]
```
 
For full changes in the colossus code: https://bitbucket.org/maximastaschov/colossus-explorefork/src/501fbba019bb0875012d0a8f41336de31c43c0b9/?at=feature%2FAdhikariGeneralized
Alternatively, most important change was the new Delta-Function:
```
def modelAdhikari14GeneralizedDeltasp(s, Om):
    """ This is a reproduction of the Adhikari14 model already implemented as an attempt to generalize it"""
    #LUT taken from the original function, can be generated through geomspace, too, but I got issues with that that I couldn't resolve yet
    c = np.array(
        [5.503e+02, 2.370e+02, 1.152e+02, 6.164e+01, 3.562e+01, 2.188e+01, 1.410e+01, 9.431e+00, 6.495e+00, 4.571e+00,
         3.266e+00, 2.354e+00, 1.701e+00, 1.223e+00, 8.671e-01, 5.971e-01, 3.896e-01, 2.283e-01, 1.014e-01, 8.349e-04])
    bins_Om = np.linspace(1.000e-01, 9.999e-01, 20, endpoint=True)
    bins_s = np.geomspace(5.000e-1, 3.999e+00, 20, endpoint=True)

    Deltasp_fitting = np.zeros((len(bins_Om), len(bins_s)))

    # Constants
    A = 38
    b = 0.57
    c_delta = 0.02
    d = 0.2
    e = 0.52

    # Loop over bins_Om and bins_s to compute the values
    for i in range(len(bins_Om)):
        for j in range(len(bins_s)):
            Deltasp_fitting[i, j] = A * (bins_Om[i] ** (-b - (c_delta * bins_s[j]))) * (np.e ** (
                    (d * bins_Om[i]) + (e * (bins_s[j] ** (3 / 4)))))

    ####This is a placeholder for the implementation of the real model instead of the fitting function####
    Deltasp = Deltasp_fitting
    ####

    Om_ = np.ones((len(s)), float) * Om
    interp = scipy.interpolate.RectBivariateSpline(bins_Om, bins_s, Deltasp)
    Delta = interp(Om_, s, grid=False)

    if np.max(s) > np.max(bins_s):
        raise Exception('Found s = %.2f, greater than max %.2f.' % (np.max(s), np.max(bins_s)))
    if np.min(s) < np.min(bins_s):
        raise Exception('Found s = %.2f, smaller than min %.2f.' % (np.min(s), np.min(bins_s)))

    c = np.interp(s, bins_s, c)
    if np.count_nonzero(c < 0.0) > 0:
        raise Exception('Found negative concentration values.')

    return Delta, c
```


# 3: So what? (What does it mean?)
## Describe your results

The resulting plot comparing the model to the fitting function shows similarities, but the difference is to big to make use of the fitting function for the further progress. 

![ComparisonModels.PNG](/img/ComparisonModels.PNG)

Still it was very helpful for understanding the structure of the colossus code. Also I looked into how to implement the model itself and struggled with the implementation of a new LUT based on the equation `\\Delta_{\\rm sp} = M_{\\rm sp} / (4 \\pi R_{\\rm sp}^3 / 3) / \\rho_{\\rm m}` found in [1]. This step is important to achieve the goal of the implementation of a generalized Delta function.



# 4. Now what? (What's next?)
## Plan for the next week

The goal now should be to solve the issue with the new model only using the fitting function. Based on that it will be possible to make changes necessery to achieve a generalized Adhikari model.

# 5. Bibliography

[1] Benedikt Diemer, *COLOSSUS: A PYTHON TOOLKIT FOR COSMOLOGY, LARGE-SCALE STRUCTURE, AND DARK MATTER HALO*, https://arxiv.org/pdf/1712.04512, Accessed: 27/01/25
[2] Adhikari et al., *Splashback in accreting dark matter halos*, https://arxiv.org/pdf/1409.4482, Accessed: 10/03/25